# 7.3 모델의 성능을 최대로 끌어올리기

## 7.3.1 고급 구조 패턴

### 배치 정규화(batch normalization)

- 지금까지 예제에서 사용해왔던 정규화는 모델에 데이터 주입 전에 한번만 정규화를 하였음
    - **주입 전에 정규화를 통해 평균이 0, 분산이 1로 맞춰 놓았지만 훈련 중에 이 분포가 유지될 것이라고 기대하기는 어려움**
    
    
- **batch normalization은 훈련 중에 데이터 분포가 바뀌더라도 이에 적응해서 데이터를 정규화 하면서 분포를 유지시키는 방법**
    - residual connection처럼 gradient의 전파를 도와줌
        - 입출력의 데이터 분포를 유지하도록 도와주므로 더 잘 전파됨
    - 더 깊은 네트워크를 구성할 수 있도록 도와줌


#### 동작 방식

- 훈련할 때
    - mini batch의 평균, 분산을 이용해 정규화
    - 1에서 얻은 평균, 분산을 Moving Average 방식으로 반복 업데이트해서, 전체 데이터에 대한 평균/분산을 추정함(테스트 시 사용하려고)
    

- 테스트할 때
    - 훈련할 때 Moving Average 방식으로 구해둔 추정된 평균/분산을 이용해 정규화함
    

- 유의할 점
    - 훈련 중에는 mini batch의 평균, 분산으로 정규화함
    - Moving Average로 추정중인 평균, 분산은 테스트할 때만 사용

#### keras의 `BatchNormalization`클래스
- `momentum` 매개변수
    - 지수이동평균 계산에 사용
    - 기본값은 0.99
        
- `axis` 매개변수
    - 정규화 할 특성 축을 지정(일반적으로 `channels` 축)
    - layer를 `data_format="channels_last"`로 사용하는 경우 -1(기본값)
        - `(batch_size, height, width, channels)`
    - layer를 `data_format="channels_first"`로 사용하는 경우 1
            - `(batch_size, channels, height, width)`


- `BatchNormalization` 층의 사용 위치
    - 일반적으로 합성곱이나 완전 연결 층 다음에 사용함

In [ ]:
# BatchNormalization 사용 위치 예

# 1. 합성곱 층 뒤
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

# 2. 완전연결층 뒤
conv_model.add(layers.Dense(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

#### 참고) 배치 재정규화(batch renormalization)

- batch normalization보다 추가적인 비용을 들이지 않고 이득을 볼 수 있는 방법
    - 저자는 batch norm을 대체할 가능성이 높다고 보고있음
    

- 자세한 내용은 이 책에서 다루지 않으므로 논문 참고
    - [Sergey Ioffe, “Batch Renormalization: Towards Reducing Minibatch Dependence in Batch-Normalized Models” (2017)](https://arxiv.org/abs/1702.03275)

#### 참고) 자기 정규화 신경망(self-normalizing neural networks)

- 특정 활성화 함수(selu)와 초기화 방법(lecun_normal)을 사용해 Dense층의 출력을 정규화하는 방법
- FCN에서 적용가능한 방법


- 자세한 내용은 이 책에서 다루지 않으므로 논문 참고
    - [Günter Klambauer et al., “Self-Normalizing Neural Networks,” Conference on Neural Information Processing Systems (2017)](https://arxiv.org/abs/1706.02515)

### 깊이별 분리 합성곱

- **깊이별 분리 합성곱(depthwise separable convolution)층은 입력 채널 별로 각각 공간 방향의 합성곱을 수행 후 1x1합성곱을 통해 출력 채널을 합치는 방법**


- 깊이별 분리 합성곱은 공간 방향 특성의 학습과 채널 방향 특성의 학습을 분리하는 효과를 냄
    - 공간상의 위치는 상관관계가 크지만 채널별로는 독립적이라고 가정하면 타당함


- 모델의 파라미터와 연산의 수를 크게 줄여줌
    - 더 작고 빠른 모델을 만들 수 있음
    - 작은 데이터로도 더 좋은 표현을 학습하고 더 성능이 좋은 모델을 만들 수 있음


- **제한된 데이터로 작은 모델을 처음부터 훈련시킬 때 특히 더 중요함**


- 깊이별 분리 합성곱의 자세한 이론적 배경 및 내용은 필자의 논문 참고
    - [Francois Chollet, "Xception: Deep Learning with Depthwise
Separable Convolutions" (2016)](https://arxiv.org/abs/1610.02357)

In [1]:
# 깊이별 분리 합성곱 예

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

h = 64 # height
w = 64 # width
c = 3  # channels
num_classes = 10

model = Sequential()

model.add(layers.SeparableConv2D(32, 3,
                                 activation='relu',
                                 input_shape=(h, w, c,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))

model.add(layers.Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 62, 62, 32)        155       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 60, 60, 64)        2400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 64)        0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 28, 28, 64)        4736      
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 26, 26, 128)       8896      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
separable_conv2d_4 (Separabl (None, 11, 11, 64)        9

## 7.3.2 하이퍼파라미터 최적화

- **모델의 구조와 관련된 파라미터들을 역전파로 훈련되는 모델 파라미터와 구분하여 하이퍼파라미터라고 함**

- 하이퍼파라미터의 예
    - 층의 수
    - 층의 유닛, 필터 수
    - 활성화 함수의 선택
    - Dropout의 정도
    - BatchNormalization의 위치
    


### 하이퍼 파라미터 자동 최적화
    
- 머신러닝 엔지니어와 연구자들은 하이퍼파라미터 튜닝에 대부분의 시간을 사용함
    - 이를 기계에 위임하고자 하는 것이 **하이퍼파라미터 자동 최적화**라는 연구 분야이며 중요함
    

- 하이퍼파라미터 자동 최적화의 역할
    - 가능한 결정 공간을 자동적, 조직적, 규칙적 방법으로 탐색하여야 함
    - 가능성 있는 구조를 탐색하여 실제 가장 높은 성능을 내는 모델을 찾아야 함


- 전형적인 하이퍼파라미터 최적화 과정
    1. 하이퍼파라미터를 선택
    2. 선택된 하이퍼파라미터로 모델 생성
    3. 훈련데이터로 학습, 검증 데이터로 성능 측정
    4. 다음으로 시도할 하이퍼파라미터를 선택
    5. 위 과정을 반복
    6. 마지막으로 테스트 데이터에서 성능 측정
    

- 위 과정에서 **검증 성능을 사용해 다음에 시도할 하이퍼파라미터를 선택하는 알고리즘**이 이 과정의 핵심(3~4단계)
    - 사용가능한 여러가지 기법들
        - 베이지안 최적화(bayesian optimization)
        - 유전 알고리즘(genetic algorithms)
        - 간단한 랜덤 탐색(random search)
        
        
#### 하이퍼파라미터 업데이트가 어려운 이유
1. 피드백 신호의 계산은 비용이 매우 많이 듦
    - 새로운 모델을 만들고 다시 처음부터 훈련해야 함
        
        
2. 하이퍼파라미터 공간은 일반적으로 분리된 결정을로 채워짐
    - 연속적이지 않으므로 미분이 불가능 (경사하강법을 적용할 수 없음)
    - gradient-free 최적화 기법을 사용해야 함 (경사하강법보다 비효율적)


#### 하이퍼파라미터 최적화 도구들 (분야가 어렵고 초창기이므로 사용가능한 도구가 적음)
- 랜덤 탐색
    - 반복적으로 랜덤하게 하이퍼파라미터를 선택하는 방법
    
- 파이썬 라이브러리 [Hyperopt](https://github.com/hyperopt/hyperopt)
    - Parzen 트리 추정기를 사용해 잘 동작할 것 같은 하이퍼파라미터 조합을 예측
    
- [Hyperas](https://github.com/maxpumperla/hyperas)
    - Hyperopt와 연동해서 케라스 모델에서 사용할 수 있음
    
    
- 참고) 대규모로 자동화하여 하이퍼파라미터 최적화를 진행할 때 검증 데이터에 과대적합되는 것을 항상 기억해야 함
    
#### 얕은학습, 깊은학습, Feature Engineering, 하이퍼파라미터 최적화

- 앝은 학습
    - 모델에 넣을 특성을 사람이 직접 만드는 방식
    - feature engineering이 수작업이므로 최적화되지 않은 방법
    
    
- 깊은 학습(딥러닝)
    - 피드백 신호를 사용해 특성이 학습되는 방식
    - feature engineering이 자동화되므로 최적화 할 수 있으며 최적화 하는것이 당연함
        - **하이퍼파라미터 최적화 분야가 크게 성장할 가능성이 높음**

## 7.3.3 모델 앙상블

### 모델 앙상블
- **모델 앙상블(model ensemble)은 여러 다른 모델의 예측을 합쳐서 더 좋은 예측을 만드는 기법**


### 앙상블 방법
1. 각 모델들의 예측을 평균내서 추론
    - 각 분류기들이 어느정도 비슷하게 좋을 때 잘 작동함
    - 분류기 중 하나가 다른 모델보다 월등히 나쁘면 앙상블의 예측이 앙상블에 있는 가장 좋은 분류기의 예측보다 좋지 않음
        
~~~
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)
~~~


2. 검증 데이터에서 학습된 가중치를 사용해 가중 평균내서 추론
    - 좋은 분류기에 높은 가중치, 안좋은 분류기에 낮은 가중치를 적용
    - 앙상블 가중치를 찾기 위한 간단한 최적화 알고리즘으로는 넬더-미드(Nelder-Mead) 방법 등이 있음
    
~~~
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

# 각 가중치(0.5, 0.25, 0.1, 0.15)는 경험적으로 학습되었다고 가정
final_preds = 0.5*preds_a + 0.25*preds_b + 0.1*preds_c + 0.15*preds_d
~~~


3. 기타 여러가지 변종들
    - 검증 데이터에서 찾은 최적의 가중치로 단순하게 가중 평균하는 방법 등
    
    
### 앙상블의 필수 요소 : 다양성

- 앙상블이 잘 동작하기 위한 핵심은 **분류기의 다양성**
    - 같은 네트워크로 초기화만 다르게 하는 것은 앙상블 할 가치가 없음
    - 가능한 최대한 다르면서 좋은 모델을 앙상블 해야함
        - 매우 다른 구조를 가지거나 다른 종류의 머신러닝 방법
    - **최상의 모델이 얼마나 좋은지보다 앙상블에 사용되는 모델들이 얼마나 다양한지가 중요함**


- 실전에서 매우 성공적으로 사용되는 기본 앙상블 스타일
    - **딥러닝 모델 + 얕은 학습 모델**

    - 참고) 트리 기반 모델(random forest나 gradient boosting tree)과 딥러닝 모델을 앙상블 할 때 실전에서 잘 동작한다고 함

## 7.3.4 정리

### 1. 고성능 심층 컨브넷

#### 고성능 심층 컨브넷을 만들기 위해서는 다음의 기법들을 사용해야 함
- residual connection
- batch normalization
- depthwise separable convolution(깊이별 분리 합성곱)
    
#### 미래에는 깊이별 분리 합성곱이 일반 합성곱을 대체
- 1D, 2D, 3D, 적용 분야에 상관없이 아주 효율적으로 표현을 학습함

### 2. 하이퍼파라미터 최적화

#### 하이퍼파라미터는 랜덤한 선택보다 하이퍼파라미터 공간을 조직적으로 탐색하는 것이 좋음
- Hyperopt와 Hyperas 라이브러리가 도움이 됨

#### 하이퍼파라미터 최적화 시 검증 데이터에 overfit된다는 것을 항상 염두해야 함

### 3. 앙상블

#### 최적화가 잘 된 가중 평균으로 만든 앙상블은 보통 좋은 결과를 만듦

#### 가능한 서로 다른 모델로 앙상블을 하는 것이 중요 (다양성)
- 앙상블에 사용할 각 모델은 가능한 예측 성능이 높은 것 중에서 고름